In [ ]:
%load_ext aiida
%aiida

import urllib.parse

from surfaces_tools.widgets.pdos import PdosOverlapViewerWidget

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
w = PdosOverlapViewerWidget()
url = urllib.parse.urlsplit(jupyter_notebook_url)
reference = urllib.parse.parse_qs(url.query)["pk"][0]
w.load_data(reference=int(reference))
display(w)